In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [16]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [22]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.TwoStageAutoEncoder import TwoStageAutoEncoder

In [6]:
import matplotlib.pyplot as plt

In [7]:
# from mask_generation import masked_sketch

In [23]:
batch_size = 128
latent_dims = 64
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 1000
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.000043
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=8
dense_hidden2=4
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "two_stage_small_obj_latent"
variational=False

In [10]:
batch_train_loader, batch_val_loader = data_loading.load_data(obj_data_postfix = '_obj_boundary',
                                                              part_data_post_fix = '_scaled',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

C:\Users\user\miniconda3\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [11]:
import gc
gc.collect()

21

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [24]:
reconstruction_loss_arr = []
kl_loss_obj_arr = []
kl_loss_part_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200,400], gamma=0.75)
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    IOU_weight_delta = torch.tensor([(1+epoch)/nb_epochs])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true[:, :num_classes]
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        adj_true = train_data.edge_index
        batch = train_data.batch
        
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, X_obj_true_transformed, label_true , class_true, False)
        
        node_data_pred = output[0]
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred)*torch.tensor([0.5]).cuda()),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        kl_weight = klw[icoef]
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss*10 + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            
            
#     scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_obj_arr.append(batch_kl_loss_obj.detach().item()/(i+1))
    kl_loss_part_arr.append(batch_kl_loss_part.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = torch.flatten(y_true[:, :num_classes])
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_true, X_obj_true_transformed, label_true , class_true, False)
        
        node_data_pred = output[0]
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred)*torch.tensor([0.5]).cuda()),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss*kl_weight + (obj_bbox_loss*10 + bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (obj_bbox_loss + bbox_loss + node_loss)*num_nodes
            
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
#     if kl_loss_arr[-1]>0.5 and abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.02 and bbox_loss_arr[-1]<1.3 and epoch>60:
#         icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

200
[1,   399] loss: 162.122
200
[2,   399] loss: 121.175
200
[3,   399] loss: 109.571
200
[4,   399] loss: 104.910
200
[5,   399] loss: 103.032
200
[6,   399] loss: 101.965
200
[7,   399] loss: 101.229
200
[8,   399] loss: 100.622
200
[9,   399] loss: 100.096
200
[10,   399] loss: 99.616
200
[11,   399] loss: 99.100
200
[12,   399] loss: 98.745
200
[13,   399] loss: 98.516
200
[14,   399] loss: 98.422
200
[15,   399] loss: 98.231
200
[16,   399] loss: 98.023
200
[17,   399] loss: 97.838
200
[18,   399] loss: 97.646
200
[19,   399] loss: 97.450
200
[20,   399] loss: 97.258
200
[21,   399] loss: 97.040
200
[22,   399] loss: 96.815
200
[23,   399] loss: 96.560
200
[24,   399] loss: 96.289
200
[25,   399] loss: 96.015
200
[26,   399] loss: 95.706
200
[27,   399] loss: 95.376
200
[28,   399] loss: 95.054
200
[29,   399] loss: 94.702
200
[30,   399] loss: 94.321
200
[31,   399] loss: 93.917
200
[32,   399] loss: 93.442
200
[33,   399] loss: 93.015
200
[34,   399] loss: 92.588
200
[35,   399

200
[278,   399] loss: 12.289
200
[279,   399] loss: 12.249
200
[280,   399] loss: 12.209
200
[281,   399] loss: 12.169
200
[282,   399] loss: 12.129
200
[283,   399] loss: 12.090
200
[284,   399] loss: 12.050
200
[285,   399] loss: 12.011
200
[286,   399] loss: 11.972
200
[287,   399] loss: 11.933
200
[288,   399] loss: 11.894
200
[289,   399] loss: 11.856
200
[290,   399] loss: 11.817
200
[291,   399] loss: 11.779
200
[292,   399] loss: 11.741
200
[293,   399] loss: 11.704
200
[294,   399] loss: 11.666
200
[295,   399] loss: 11.629
200
[296,   399] loss: 11.592
200
[297,   399] loss: 11.555
200
[298,   399] loss: 11.519
200
[299,   399] loss: 11.482
200
[300,   399] loss: 11.446
200
[301,   399] loss: 11.410
200
[302,   399] loss: 11.374
200
[303,   399] loss: 11.339
200
[304,   399] loss: 11.304
200
[305,   399] loss: 11.269
200
[306,   399] loss: 11.234
200
[307,   399] loss: 11.200
200
[308,   399] loss: 11.165
200
[309,   399] loss: 11.131
200
[310,   399] loss: 11.097
200
[311, 

[558,   399] loss: 6.297
200
[559,   399] loss: 6.285
200
[560,   399] loss: 6.274
200
[561,   399] loss: 6.262
200
[562,   399] loss: 6.251
200
[563,   399] loss: 6.239
200
[564,   399] loss: 6.228
200
[565,   399] loss: 6.217
200
[566,   399] loss: 6.205
200
[567,   399] loss: 6.194
200
[568,   399] loss: 6.183
200
[569,   399] loss: 6.172
200
[570,   399] loss: 6.161
200
[571,   399] loss: 6.150
200
[572,   399] loss: 6.138
200
[573,   399] loss: 6.127
200
[574,   399] loss: 6.116
200
[575,   399] loss: 6.105
200
[576,   399] loss: 6.094
200
[577,   399] loss: 6.084
200
[578,   399] loss: 6.073
200
[579,   399] loss: 6.062
200
[580,   399] loss: 6.051
200
[581,   399] loss: 6.041
200
[582,   399] loss: 6.030
200
[583,   399] loss: 6.019
200
[584,   399] loss: 6.009
200
[585,   399] loss: 5.998
200
[586,   399] loss: 5.988
200
[587,   399] loss: 5.977
200
[588,   399] loss: 5.967
200
[589,   399] loss: 5.956
200
[590,   399] loss: 5.946
200
[591,   399] loss: 5.936
200
[592,   399] l

200
[841,   399] loss: 4.119
200
[842,   399] loss: 4.114
200
[843,   399] loss: 4.109
200
[844,   399] loss: 4.104
200
[845,   399] loss: 4.099
200
[846,   399] loss: 4.094
200
[847,   399] loss: 4.089
200
[848,   399] loss: 4.085
200
[849,   399] loss: 4.080
200
[850,   399] loss: 4.075
200
[851,   399] loss: 4.070
200
[852,   399] loss: 4.065
200
[853,   399] loss: 4.060
200
[854,   399] loss: 4.055
200
[855,   399] loss: 4.051
200
[856,   399] loss: 4.046
200
[857,   399] loss: 4.042
200
[858,   399] loss: 4.037
200
[859,   399] loss: 4.032
200
[860,   399] loss: 4.027
200
[861,   399] loss: 4.022
200
[862,   399] loss: 4.018
200
[863,   399] loss: 4.012
200
[864,   399] loss: 4.008
200
[865,   399] loss: 4.003
200
[866,   399] loss: 3.998
200
[867,   399] loss: 3.994
200
[868,   399] loss: 3.989
200
[869,   399] loss: 3.985
200
[870,   399] loss: 3.980
200
[871,   399] loss: 3.976
200
[872,   399] loss: 3.972
200
[873,   399] loss: 3.967
200
[874,   399] loss: 3.962
200
[875,   39

In [26]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_true, X_obj_true_transformed, label_true , class_true)
    node_data_pred_test = output[0]
    X_obj_pred_test = output[1]
#     node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    X_obj_pred_test = torch.cat(((torch.tensor([1.0]).cuda()-X_obj_pred_test)*torch.tensor([0.5]).cuda(),
                                (torch.tensor([1.0]).cuda()+X_obj_pred_test)*torch.tensor([0.5]).cuda()),
                              -1)
    res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("cpu").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("cpu").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

In [27]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',
                                                                   'IOU', 'MSE']]

,obj_class,part_labels,IOU,MSE
0,bird,head,0.415618,0.001603
1,bird,lfoot,0.167955,0.000925
2,bird,lleg,0.263575,0.000777
3,bird,lwing,0.287303,0.002730
4,bird,neck,0.265450,0.001751
...,...,...,...,...
81,sheep,lhorn,0.289651,0.001233
82,sheep,neck,0.466348,0.000712
83,sheep,rhorn,0.197374,0.001557
84,sheep,tail,0.120423,0.001762


In [28]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

,obj_class,IOU,MSE
0,bird,0.384084,0.001544
1,cat,0.513757,0.002798
2,cow,0.431734,0.001010
3,dog,0.360434,0.002096
4,horse,0.343948,0.001739
5,person,0.417685,0.001664
6,sheep,0.486532,0.001444


In [29]:
res_obj_level[['IOU', 'MSE']].mean()

IOU    0.416270
MSE    0.001856
dtype: float64

In [30]:
X_obj_pred[:10,2:]-X_obj_pred[:10,:2]

tensor([[0.3849, 0.3379],
        [0.5016, 0.6741],
        [0.2205, 0.1853],
        [0.3917, 0.4132],
        [0.3243, 0.3643],
        [0.1673, 0.1172],
        [0.2245, 0.1029],
        [0.2815, 0.2084],
        [0.3308, 0.3412],
        [0.4317, 0.4058]], grad_fn=<SubBackward0>)

In [31]:
X_obj_true[:10, 2:]-X_obj_true[:10, :2]

tensor([[0.5268, 0.3339],
        [0.4683, 0.6751],
        [0.3386, 0.1817],
        [0.4905, 0.4107],
        [0.4924, 0.3591],
        [0.2409, 0.1152],
        [0.1879, 0.1041],
        [0.2323, 0.2100],
        [0.4550, 0.3370],
        [0.2951, 0.4102]])

In [ ]:
res_obj_level.to_csv(model_path+'/obj_metrics.csv')

In [26]:
result.mean().reset_index()

C:\Users\user\AppData\Local\Temp/ipykernel_21444/2602046386.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  result.mean().reset_index()


,index,0
0,Unnamed: 0,1018.623810
1,IOU,0.339382
2,MSE,0.002054
3,id,3359.500000


In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)